# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
raw_data = pd.read_excel('Dataset1 .xlsx')
raw_data.sample(5)

,emg1,emg2,emg3,emg4,emg5,emg6,emg7,emg8,emg9,emg10,repetition,rerepetition,stimulus,restimulus
91660,0.0024,0.0024,0.0024,0.0024,0.0024,0.0024,0.0024,0.0342,0.0024,0.0024,0,0,0,0
69207,0.0024,0.0024,0.0024,0.0024,0.0024,0.0024,0.0854,0.0806,0.0024,0.0146,3,0,9,0
23800,0.0024,0.0024,0.0024,0.0024,0.0024,0.0024,0.0024,0.0464,0.0024,0.0049,0,0,0,0
45798,0.0684,0.7202,0.3662,0.1367,0.0049,0.0024,0.5151,0.2954,0.4224,0.5493,5,5,6,6
21383,0.0049,0.0024,0.0806,0.0195,0.0024,0.0024,0.0024,0.0708,0.1196,0.0049,6,6,3,3


In [3]:
index_names = raw_data[ (raw_data['stimulus'] != raw_data['restimulus'])].index
raw_data.drop(index_names, inplace = True)

In [4]:
index_names = raw_data[ (raw_data['repetition'] != raw_data['rerepetition'])].index
raw_data.drop(index_names, inplace = True)

In [5]:
X = raw_data.iloc[:,0:10]
y = raw_data.stimulus

In [6]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from tensorflow.keras import activations

In [7]:
num_classes = 13

In [8]:
# One hot encoding for categorical labels
import keras
y = keras.utils.to_categorical(y, num_classes)

In [9]:
# build the model
input_dim = X.shape[1]

In [10]:
from sklearn.preprocessing import StandardScaler
standardscaler=StandardScaler()

In [11]:
sc = pd.DataFrame(standardscaler.fit_transform(X))

In [12]:
sc

,0,1,2,3,4,5,6,7,8,9
0,0.124694,-0.428949,-0.447634,-0.376741,-0.171824,0.139046,-0.439076,-0.441237,-0.537378,-0.250667
1,0.057307,-0.428949,-0.447634,-0.376741,-0.171824,-0.060491,-0.439076,-0.478142,-0.537378,-0.289053
2,0.040804,-0.428949,-0.447634,-0.376741,-0.171824,-0.060491,-0.439076,-0.490275,-0.537378,-0.327440
3,0.040804,-0.428949,-0.447634,-0.376741,-0.171824,-0.060491,-0.439076,-0.515047,-0.537378,-0.384629
4,-0.010081,-0.428949,-0.447634,-0.376741,-0.171824,-0.162295,-0.439076,-0.502914,-0.537378,-0.404214
...,...,...,...,...,...,...,...,...,...,...
75038,-0.329139,-0.428949,-0.447634,-0.376741,0.048955,-0.162295,-0.439076,-0.502914,-0.537378,-0.499790
75039,-0.329139,-0.428949,-0.447634,-0.376741,-0.171824,-0.162295,-0.439076,-0.515047,-0.537378,-0.499790
75040,-0.329139,-0.428949,-0.447634,-0.376741,-0.171824,-0.162295,-0.439076,-0.515047,-0.537378,-0.461403
75041,-0.329139,-0.428949,-0.447634,-0.376741,-0.171824,-0.162295,-0.439076,-0.502914,-0.537378,-0.461403


In [13]:
# Splitting data for training and testing
X_train, X_test, y_train, y_test = train_test_split(sc, y, test_size = 0.2, random_state = 39)

In [14]:

import tensorflow as tf
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dropout


In [15]:
import numpy as np
from matplotlib import pyplot as plt

from sklearn.model_selection import GridSearchCV

In [16]:
from keras.optimizers import SGD

In [17]:
def define_model(learning_rate=0.01, momentum=0.1):   
    
    
    visible = Input(shape=(input_dim,))
    hidden1 = Dense(10, activation='relu')(visible)
    hidden2 = Dense(20, activation='relu')(hidden1)
    hidden3 = Dense(10, activation='relu')(hidden2)
    output = Dense(num_classes, activation='softmax')(hidden3)
    model = Model(inputs=visible, outputs=output)
    
    
    
    
    # compile the model
    optimizer = SGD(lr=learning_rate, momentum=momentum)
    model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['acc'])
    return model

In [18]:
from keras.wrappers.scikit_learn import KerasClassifier
batch_size = 10
epochs = 10

model = KerasClassifier(build_fn=define_model, epochs=epochs, batch_size = batch_size)

learning_rate = [0.0001, 0.001, 0.01, 0.1]
momentum = [0.3, 0.5, 0.7, 0.9]


#Cross validation, cv=3
param_grid = dict(learning_rate=learning_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=3)

In [19]:
grid_result = grid.fit(X_train, y_train)

Epoch 1/10
4003/4003 [==============================] - 5s 1ms/step - loss: 2.5045 - acc: 0.2384
Epoch 2/10
4003/4003 [==============================] - 4s 1ms/step - loss: 2.1881 - acc: 0.5111A: 1s - loss: 2
Epoch 3/10
4003/4003 [==============================] - 4s 1ms/step - loss: 1.9158 - acc: 0.5116
Epoch 4/10
4003/4003 [==============================] - 5s 1ms/step - loss: 1.7622 - acc: 0.5126
Epoch 5/10
4003/4003 [==============================] - 4s 1ms/step - loss: 1.7111 - acc: 0.5134
Epoch 6/10
4003/4003 [==============================] - 5s 1ms/step - loss: 1.6805 - acc: 0.5156
Epoch 7/10
4003/4003 [==============================] - 5s 1ms/step - loss: 1.6553 - acc: 0.5186
Epoch 8/10
4003/4003 [==============================] - ETA: 0s - loss: 1.6342 - acc: 0.5215- E - 4s 1ms/step - loss: 1.6338 - acc: 0.5215
Epoch 9/10
4003/4003 [==============================] - 4s 1ms/step - loss: 1.6153 - acc: 0.5237
Epoch 10/10
2002/2002 [==============================] - 2s 927us/step

4003/4003 [==============================] - 4s 913us/step - loss: 1.1435 - acc: 0.6666
Epoch 8/10
4003/4003 [==============================] - 4s 930us/step - loss: 1.0801 - acc: 0.6845
Epoch 9/10
4003/4003 [==============================] - 4s 946us/step - loss: 1.0204 - acc: 0.7061
Epoch 10/10
2002/2002 [==============================] - 2s 919us/step - loss: 0.9455 - acc: 0.7301
Epoch 1/10
4003/4003 [==============================] - 4s 945us/step - loss: 1.6063 - acc: 0.5492
Epoch 2/10
4003/4003 [==============================] - 4s 1ms/step - loss: 1.3257 - acc: 0.6080
Epoch 3/10
4003/4003 [==============================] - 5s 1ms/step - loss: 1.2069 - acc: 0.6361
Epoch 4/10
4003/4003 [==============================] - 6s 1ms/step - loss: 1.1073 - acc: 0.6720A: 0s - loss: 1.1073 - acc:
Epoch 5/10
4003/4003 [==============================] - 5s 1ms/step - loss: 1.0077 - acc: 0.7060A: 3s - loss: - ETA: 1s - loss
Epoch 6/10
4003/4003 [==============================] - 5s 1ms/step - 

4003/4003 [==============================] - 5s 1ms/step - loss: 0.6474 - acc: 0.8146
Epoch 5/10
4003/4003 [==============================] - 5s 1ms/step - loss: 0.6113 - acc: 0.8220A:  - ETA: 0s - loss: 0.6111 - acc: 0.82
Epoch 6/10
4003/4003 [==============================] - 5s 1ms/step - loss: 0.5870 - acc: 0.8277A: 0s - loss: 0.5
Epoch 7/10
4003/4003 [==============================] - 5s 1ms/step - loss: 0.5693 - acc: 0.8334A: 4s - loss: 0.5731 - a - ETA: 0s - loss: 0.5696 - acc: 0.8
Epoch 8/10
4003/4003 [==============================] - 5s 1ms/step - loss: 0.5582 - acc: 0.8341A: 0s - loss: 0.
Epoch 9/10
4003/4003 [==============================] - 5s 1ms/step - loss: 0.5498 - acc: 0.8386
Epoch 10/10
2002/2002 [==============================] - 3s 1ms/step - loss: 0.5735 - acc: 0.8285
Epoch 1/10
4003/4003 [==============================] - 5s 1ms/step - loss: 1.1168 - acc: 0.6770
Epoch 2/10
4003/4003 [==============================] - 6s 1ms/step - loss: 0.7453 - acc: 0.7877
Epoc

4003/4003 [==============================] - 8s 2ms/step - loss: 0.8518 - acc: 0.7560
Epoch 2/10
4003/4003 [==============================] - 8s 2ms/step - loss: 0.7067 - acc: 0.7948A: 2s - loss: 0.7047 - acc: 0. - ETA: 1s - l - ETA: 0s - loss: 0.7071 -
Epoch 3/10
4003/4003 [==============================] - 7s 2ms/step - loss: 0.6801 - acc: 0.8019A: 5s - lo - E
Epoch 4/10
4003/4003 [==============================] - 7s 2ms/step - loss: 0.6708 - acc: 0.8048A: 2s - los
Epoch 5/10
4003/4003 [==============================] - 7s 2ms/step - loss: 0.6813 - acc: 0.8024A: - ETA: 1s - loss: 
Epoch 6/10
4003/4003 [==============================] - 7s 2ms/step - loss: 0.6596 - acc: 0.8068
Epoch 7/10
4003/4003 [==============================] - 8s 2ms/step - loss: 0.6524 - acc: 0.8078
Epoch 8/10
4003/4003 [==============================] - 8s 2ms/step - loss: 0.6924 - acc: 0.8000
Epoch 9/10
4003/4003 [==============================] - 6s 2ms/step - loss: 0.6640 - acc: 0.8071ETA: 0s - loss: 0.6682

4003/4003 [==============================] - 4s 970us/step - loss: 1.9335 - acc: 0.5066
Epoch 9/10
4003/4003 [==============================] - 5s 1ms/step - loss: 1.9325 - acc: 0.5066
Epoch 10/10
2002/2002 [==============================] - 2s 972us/step - loss: 1.9125 - acc: 0.5138
Epoch 1/10
6004/6004 [==============================] - 6s 1ms/step - loss: 0.8694 - acc: 0.7493
Epoch 2/10
6004/6004 [==============================] - 8s 1ms/step - loss: 0.6512 - acc: 0.8107
Epoch 3/10
6004/6004 [==============================] - 7s 1ms/step - loss: 0.6077 - acc: 0.8212
Epoch 4/10
6004/6004 [==============================] - 6s 1ms/step - loss: 0.5844 - acc: 0.8276A: 4s - loss: - ETA: 3s - 
Epoch 5/10
6004/6004 [==============================] - 7s 1ms/step - loss: 0.5687 - acc: 0.8307
Epoch 6/10
6004/6004 [==============================] - 6s 995us/step - loss: 0.5580 - acc: 0.8336
Epoch 7/10
6004/6004 [==============================] - 6s 948us/step - loss: 0.5488 - acc: 0.8365
Epoch 

In [20]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean = %f (std=%f) with: %r" % (mean, stdev, param))


Best: 0.838192 using {'learning_rate': 0.01, 'momentum': 0.7}
Mean = 0.531715 (std=0.006737) with: {'learning_rate': 0.0001, 'momentum': 0.3}
Mean = 0.551521 (std=0.013620) with: {'learning_rate': 0.0001, 'momentum': 0.5}
Mean = 0.608156 (std=0.028607) with: {'learning_rate': 0.0001, 'momentum': 0.7}
Mean = 0.676866 (std=0.027423) with: {'learning_rate': 0.0001, 'momentum': 0.9}
Mean = 0.739348 (std=0.019128) with: {'learning_rate': 0.001, 'momentum': 0.3}
Mean = 0.778742 (std=0.011754) with: {'learning_rate': 0.001, 'momentum': 0.5}
Mean = 0.802512 (std=0.005583) with: {'learning_rate': 0.001, 'momentum': 0.7}
Mean = 0.827897 (std=0.003887) with: {'learning_rate': 0.001, 'momentum': 0.9}
Mean = 0.830013 (std=0.006699) with: {'learning_rate': 0.01, 'momentum': 0.3}
Mean = 0.834111 (std=0.005089) with: {'learning_rate': 0.01, 'momentum': 0.5}
Mean = 0.838192 (std=0.003991) with: {'learning_rate': 0.01, 'momentum': 0.7}
Mean = 0.821085 (std=0.004895) with: {'learning_rate': 0.01, 'moment